將資料檔 HealthCheck.csv 讀入

In [91]:
import pandas as pd
import numpy as np
import os

Dataset_File = "HealthCheck.csv"
if not os.path.isfile(Dataset_File):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" + Dataset_File)

In [92]:
dataset = pd.read_csv("HealthCheck.csv")
print(dataset)

      Race  Height_cm  Weight_kg Over70yr
0    White      186.0       90.0      Yes
1  African      185.0       98.0       No
2    Asian      175.0       80.0       No
3  African      170.0       75.0      Yes
4    Asian      164.0        NaN       No
5    Asian      170.0       72.0      Yes
6    White      178.0       75.0       No
7    White        NaN       89.0      Yes
8  African      165.0       79.0      Yes


將資料集切分為「自變數 X」與「應變數 Y」

In [93]:
X = dataset.iloc[:, :-1].values #自變數(直行:從最左到倒數第二行)
Y = dataset.iloc[:, 3].values #應變數(直行:最後一行yes,no那行)

把缺失資料補上(取欄平均填入)

In [94]:
from sklearn.impute import SimpleImputer #填補遺失資料用的

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer = imputer.fit(X[:, 1:3]) #fit根據「陣列切片」出來的結果，去計算平均
X[:, 1:3] = imputer.transform(X[:, 1:3]) #transform將陣列切片內的「缺失資料」，轉化為計算出來的值

將資料集裡面的文字欄位，包含自變數 X 與應變數 Y，轉換為數字欄位


In [95]:
#使用標籤編碼器(LabelEncoder) 對 Y 做類別資料數位化
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
Y = labelEncoder.fit_transform(Y).astype("float64")

In [96]:
#使用One-Hot Encoder對 X 做類別資料數位化
array_dummies = pd.get_dummies(X[:, 0]).values
X = np.concatenate((array_dummies, X[:, 1:3]), axis=1).astype("float64")

將資料集切分為「訓練集」與「測試集」

In [102]:
from sklearn.model_selection import train_test_split #切分「訓練集、測試集」使用

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("自變數訓練集\n", X_train)
print("自變數測試集\n", X_test)
print("應變數訓練集", Y_train)
print("應變數測試集", Y_test)

自變數訓練集
 [[  1.     0.     0.   185.    98.  ]
 [  0.     1.     0.   164.    82.25]
 [  1.     0.     0.   165.    79.  ]
 [  0.     0.     1.   178.    75.  ]
 [  1.     0.     0.   170.    75.  ]
 [  0.     0.     1.   186.    90.  ]
 [  0.     1.     0.   170.    72.  ]]
自變數測試集
 [[  0.      0.      1.    174.125  89.   ]
 [  0.      1.      0.    175.     80.   ]]
應變數訓練集 [0. 0. 1. 0. 1. 1. 1.]
應變數測試集 [1. 0.]


針對自變數 X 相關資料集，進行「特徵縮放」

In [103]:
from sklearn.preprocessing import StandardScaler #使用「標準縮放器（StandardScaler）」物件

sc_X = StandardScaler().fit(X_train)
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)

In [104]:
print("特徵縮放後自變數訓練集\n", X_train)
print("特徵縮放後自變數測試集\n", X_test)
print("特徵縮放後應變數訓練集", Y_train)
print("特徵縮放後應變數測試集", Y_test)

特徵縮放後自變數訓練集
 [[ 1.15470054 -0.63245553 -0.63245553  1.30941777  1.89111602]
 [-0.8660254   1.58113883 -0.63245553 -1.19037979  0.07416141]
 [ 1.15470054 -0.63245553 -0.63245553 -1.07134181 -0.30076573]
 [-0.8660254  -0.63245553  1.58113883  0.47615192 -0.76221452]
 [ 1.15470054 -0.63245553 -0.63245553 -0.47615192 -0.76221452]
 [-0.8660254  -0.63245553  1.58113883  1.42845575  0.96821844]
 [-0.8660254   1.58113883 -0.63245553 -0.47615192 -1.10830111]]
特徵縮放後自變數測試集
 [[-0.8660254  -0.63245553  1.58113883  0.01487975  0.85285625]
 [-0.8660254   1.58113883 -0.63245553  0.11903798 -0.18540353]]
特徵縮放後應變數訓練集 [0. 0. 1. 0. 1. 1. 1.]
特徵縮放後應變數測試集 [1. 0.]
